# 各种分布
1. 字数分布：字数区间占比环形图（如50万以下/50-100万/100万以上）
2. 类型分布：小说类型占比饼图
3. 分级统计：各类型平均字数对比水平条形图
4. 评分分布：绘制正态分布曲线

In [1]:
from pymongo import MongoClient
# 连接到MongoDB
client = MongoClient('mongodb://localhost:27017/')
# 打开learn数据库
db = client['learn']
# 在数据库中打开seven_cats集合
books = db['seven_cats']

In [2]:
import pandas as pd
# 将MongoDB数据转换为pandas数据
add = pd.DataFrame(list(books.find({}, {'_id': 0, 'wordcount': 1, 'type': 1, 'serialised': 1, 'score': 1})))
add['count'] = 1
add

,score,serialised,type,wordcount,count
0,9.3,连载中,都市,272.47,1
1,9.1,连载中,武侠仙侠,38.60,1
2,8.8,连载中,玄幻奇幻,61.96,1
3,9.3,连载中,玄幻奇幻,664.36,1
4,9.1,连载中,玄幻奇幻,552.87,1
...,...,...,...,...,...
7713,9.3,完结,科幻,329.66,1
7714,8.9,完结,科幻,424.60,1
7715,9.1,连载中,都市,135.02,1
7716,9.0,完结,N次元,82.74,1


In [3]:
# 导入可视化工具
from pyecharts import options as opts

## 一、字数分布：字数区间占比环形图
（10万以下/10-30万/30-50万/50-100万/100-200万/200-300万/300-500万/500万以上）

In [4]:
# 分箱分组
agg = pd.concat([add['wordcount'], add['count']], axis=1)
bins = [-float('inf'), 10, 30, 50, 100, 200, 300, 500, float('inf')]
labels = ['10万以下', '10-30万', '30-50万', '50-100万', '100-200万', '200-300万', '300-500万', '500万以上']
agg['word_interval'] = pd.cut(agg['wordcount'], bins=bins, labels=labels)
grouped = agg.groupby('word_interval', observed=False)['count'].sum()
grouped

word_interval
10万以下        370
10-30万      1551
30-50万       938
50-100万     1277
100-200万    1873
200-300万     706
300-500万     593
500万以上       410
Name: count, dtype: int64

In [5]:
# 构建环形图
from pyecharts.charts import Pie
pie = Pie()
pie.add(
        "",
        [list(z) for z in zip(grouped.index, grouped)],
        radius=["40%", "70%"],
)
pie.set_global_opts(
        title_opts=opts.TitleOpts(title="七猫男生原创小说字数分布环形图", pos_left="center", pos_top="1%"), 
        legend_opts=opts.LegendOpts(orient="vertical", pos_top="20%", pos_left="5%", item_gap=20),
)
pie.set_series_opts(label_opts=opts.LabelOpts(color="auto", font_size=14))
pie.render("D:\sevencats-novels-analysis\images\七猫男生原创小说字数分布.html")

'D:\\sevencats-novels-analysis\\images\\七猫男生原创小说字数分布.html'

## 二、类型分布：小说类型占比饼图

In [6]:
# 分组聚合
agg = pd.concat([add['type'], add['count']], axis=1)
grouped = agg.groupby('type', observed=False)['count'].sum()
grouped

type
N次元      438
体育        75
军事        37
历史      1043
奇闻异事     237
武侠仙侠     500
游戏       135
玄幻奇幻    1812
现实题材      23
科幻       336
都市      3082
Name: count, dtype: int64

In [7]:
# 构建饼图
from pyecharts.charts import Pie
pie = Pie()
pie.add(
        "",
        [list(z) for z in zip(grouped.index, grouped)],
)
pie.set_global_opts(
        title_opts=opts.TitleOpts(title="七猫男生原创小说类型分布饼图", pos_left="center", pos_top="1%"), 
        legend_opts=opts.LegendOpts(orient="vertical", pos_top="14%", pos_left="10%", item_gap=20),
)
pie.set_series_opts(label_opts=opts.LabelOpts(color="auto", font_size=14))
pie.render("D:\sevencats-novels-analysis\images\七猫男生原创小说类型分布.html")

'D:\\sevencats-novels-analysis\\images\\七猫男生原创小说类型分布.html'

# 三、分级统计：各类型平均字数对比水平条形图

In [8]:
# 分组聚合
agg = pd.concat([add['type'], add['wordcount']], axis=1)
grouped = agg.groupby('type', observed=False)['wordcount'].mean()
sorting = grouped.astype(int).sort_values()
sorting

type
现实题材     38
历史      106
奇闻异事    115
都市      134
N次元     137
体育      143
科幻      152
游戏      171
军事      186
玄幻奇幻    192
武侠仙侠    196
Name: wordcount, dtype: int64

In [9]:
# 构建水平条形图
from pyecharts.charts import Bar
from pyecharts.commons.utils import JsCode

# 查阅各种资料进行修改所得
# 创建渐变色函数（蓝紫渐变）
color_js = JsCode('''(params) => {
    return new echarts.graphic.LinearGradient(1, 0, 0, 0, 
    [{offset: 0, color: '#5470c6'}, {offset: 1, color: '#9a60b4'}])
}''')

bar = Bar(init_opts=opts.InitOpts(
        bg_color="#f5f7fa",  # 浅灰背景
        animation_opts=opts.AnimationOpts(
                animation_delay=500,  # 动画延迟
                animation_easing="cubicOut"  # 弹性动画
        )
))
bar.add_xaxis(sorting.index.tolist())
bar.add_yaxis(
        '', sorting.tolist(), category_gap="15%", 
        label_opts=opts.LabelOpts(
                is_show=True, 
                position='right',
                font_size=12, 
                font_family="Arial", 
                color="#333",
        ), 
        itemstyle_opts=opts.ItemStyleOpts(
                color=color_js,  # 渐变色
                border_width=1.5,
                border_color="#fff",
                opacity=0.9,
        )
)
bar.reversal_axis()
bar.set_global_opts(
        title_opts=opts.TitleOpts(title="各类型平均字数对比水平条形图", pos_left="center", pos_top="3%",),
        xaxis_opts=opts.AxisOpts(
                name="平均字数（单位：万字）", 
                name_location="center", 
                name_gap=30, 
                axislabel_opts=opts.LabelOpts(
                        font_size=12,
                )
        ),
)
bar.render("D:\sevencats-novels-analysis\images\七猫各类型平均字数对比.html")

'D:\\sevencats-novels-analysis\\images\\七猫各类型平均字数对比.html'

# 四、评分分布：绘制正态分布曲线

In [4]:
# 分组聚合
agg = pd.concat([add['score'], add['count']], axis=1)
grouped = agg.groupby('score', observed=False)['count'].sum()
grouped

score
6.1       1
6.7       1
7.0     401
7.5       1
7.7       1
7.8       1
8.0    1297
8.1       8
8.2       7
8.3       9
8.4     708
8.5      69
8.6     291
8.7     287
8.8     490
8.9     634
9.0     873
9.1     739
9.2     968
9.3     432
9.4     212
9.5     148
9.6     100
9.7      30
9.8      10
Name: count, dtype: int64

In [5]:
from pyecharts.charts import Scatter, Line
from scipy.optimize import curve_fit
import numpy as np

# 构建散点图
scatter = Scatter()
scatter.add_xaxis(grouped.index.tolist())
scatter.add_yaxis('作品数量', grouped.tolist(), symbol_size=10, label_opts=opts.LabelOpts(is_show=False))

# 询问AI进行的高斯拟合
# 构建拟合曲线
ratings = grouped.index.to_numpy()
works = grouped.values
# --------------------- 高斯拟合 ---------------------
def gaussian(x, a, mu, sigma):
    """高斯函数公式
    a: 振幅（峰值高度）
    mu: 均值（中心位置）
    sigma: 标准差（分布宽度）
    """
    return a * np.exp(-(x - mu)**2 / (2 * sigma**2))

# 执行拟合（p0为初始猜测参数：振幅、均值、标准差）
popt, pcov = curve_fit(gaussian, ratings, works, p0=[1300, 8, 0.5])
fit_curve = gaussian(ratings, *popt)  # 生成拟合曲线数据

line = Line()
line.add_xaxis(ratings.tolist())
line.add_yaxis(
        "高斯拟合",
        fit_curve.tolist(),
        is_smooth=True,
        is_symbol_show=False,
        linestyle_opts=opts.LineStyleOpts(width=3, color="red"), 
        label_opts=opts.LabelOpts(is_show=False)
)

# 组合图表
scatter.overlap(line)
scatter.set_global_opts(
        title_opts=opts.TitleOpts(title="评分分布曲线", pos_left="center", pos_bottom="1%"),
        xaxis_opts=opts.AxisOpts(
                name="评分", type_="value", splitline_opts=opts.SplitLineOpts(is_show=True), min_=6
        ),
        yaxis_opts=opts.AxisOpts(
                name="作品数量", 
                type_="value",
                axistick_opts=opts.AxisTickOpts(is_show=True),
                splitline_opts=opts.SplitLineOpts(is_show=True),
        ),
)
scatter.render('D:\sevencats-novels-analysis\images\七猫评分分布曲线.html')

'D:\\sevencats-novels-analysis\\images\\七猫评分分布曲线.html'